In [1]:
%%bash
#wget http://download.tensorflow.org/models/resnet_v1_50_2016_08_28.tar.gz
#tar -xvf resnet_v1_50_2016_08_28.tar.gz
#rm resnet_v1_50_2016_08_28.tar.gz

In [2]:
import os
import sys
import numpy as np
import tensorflow as tf
# Upgrade dask before importing contrib!
import tensorflow.contrib.slim
from tensorflow.contrib.slim.nets import resnet_v1
from common.params_inf import *
from common.utils import *

In [3]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.14.1
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [5]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    for idx, dta in yield_mb_X(data, batchsize):
        pred = sess.run(classifier, feed_dict={input_tensor: dta}).squeeze()
        out[idx*batchsize:(idx+1)*batchsize] = pred
    return out

In [6]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data(BATCH_SIZE*BATCHES_GPU)
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

(1280, 224, 224, 3) (1280, 3, 224, 224)


In [7]:
# Placeholders
checkpoint_file = 'resnet_v1_50.ckpt'
input_tensor = tf.placeholder(tf.float32, shape=(None,224,224,3), name='input_image')

# Load the model
sess = tf.Session()
arg_scope = resnet_v1.resnet_arg_scope()
with tensorflow.contrib.slim.arg_scope(arg_scope):
    # Docstring ->
    #     num_classes: Number of predicted classes for classification tasks. If None
    #  we return the features before the logit layer.
    logits, end_points = resnet_v1.resnet_v1_50(input_tensor, is_training=False)
    
saver = tf.train.Saver()
saver.restore(sess, checkpoint_file)

INFO:tensorflow:Restoring parameters from resnet_v1_50.ckpt


In [9]:
cold_start = predict_fn(logits, fake_input_data_cl, BATCH_SIZE)

In [10]:
%%time
features = predict_fn(logits, fake_input_data_cl, BATCH_SIZE)

CPU times: user 1.96 s, sys: 480 ms, total: 2.44 s
Wall time: 2.26 s


In [12]:
print("Images per second {}".format((BATCH_SIZE*BATCHES_GPU)/2.26))

Images per second 566.3716814159293
